In [ ]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("Train.csv")

In [ ]:
# Group by road conditions and count the frequency of each
road_condition_counts = df['RDSFCOND'].value_counts().reset_index()
road_condition_counts.columns = ['Road Condition', 'Frequency']

# Create a bar plot using Plotly
fig = px.bar(road_condition_counts, x='Road Condition', y='Frequency',
             title='Frequency of Road Conditions in Traffic Collisions',
             labels={'Frequency': 'Collision Count', 'Road Condition': 'Road Condition'})

# Show the plot
fig.show()

In [ ]:
# Group by state and road conditions, and count the frequency of each
state_road_condition_counts = df.groupby(['DISTRICT', 'RDSFCOND']).size().reset_index(name='Frequency')

# Create a bar plot using Plotly
fig = px.bar(state_road_condition_counts, x='DISTRICT', y='Frequency', color='RDSFCOND',
             title='Frequency of Road Conditions by State',
             labels={'Frequency': 'Collision Count', 'DISTRICT': 'State', 'RDSFCOND': 'Road Condition'},
             barmode='group')

# Show the plot
fig.show()

In [ ]:
condition_counts = df.groupby(['RDSFCOND', 'VISIBILITY']).size().reset_index(name='Frequency')

# Create a bar plot using Plotly
fig = px.bar(condition_counts, x='RDSFCOND', y='Frequency', color='VISIBILITY',
             title='Frequency of Collisions by Road Conditions and Visibility',
             labels={'Frequency': 'Collision Count', 'RDSFCOND': 'Road Condition', 'VISIBILITY': 'Visibility'},
             barmode='group')

# Show the plot
fig.show()

In [ ]:
yearly_collision_counts = df['YEAR'].value_counts().reset_index()
yearly_collision_counts.columns = ['Year', 'Collision Count']

# Sort the DataFrame by year
yearly_collision_counts = yearly_collision_counts.sort_values(by='Year')

# Create a line plot using Plotly
fig = px.bar(yearly_collision_counts, x='Year', y='Collision Count',
              title='Year-wise Occurrence of Collisions',
              labels={'Collision Count': 'Collision Count', 'Year': 'Year'})

# Show the plot
fig.show()

In [ ]:
# Group by year and state and count the occurrences of collisions
state_yearly_collision_counts = df.groupby(['DISTRICT', 'YEAR']).size().reset_index(name='Collision Count')

# Sort the DataFrame by year
state_yearly_collision_counts = state_yearly_collision_counts.sort_values(by=['DISTRICT', 'YEAR'])

# Create a grouped bar plot using Plotly
fig = px.bar(state_yearly_collision_counts, x='YEAR', y='Collision Count', color='DISTRICT',
             title='Year-wise Occurrence of Collisions by State',
             labels={'Collision Count': 'Collision Count', 'YEAR': 'Year', 'DISTRICT': 'State'},
             barmode='group')

# Show the plot
fig.show()

In [ ]:
# Group by light and visibility conditions and count the occurrences of collisions
light_visibility_counts = df.groupby(['LIGHT', 'VISIBILITY']).size().reset_index(name='Collision Count')

# Create a grouped bar plot using Plotly
fig = px.bar(light_visibility_counts, x='LIGHT', y='Collision Count', color='VISIBILITY',
             title='Comparison of Light and Visibility Conditions in Collisions',
             labels={'Collision Count': 'Collision Count', 'LIGHT': 'Light Condition', 'VISIBILITY': 'Visibility Condition'},
             barmode='group')

# Show the plot
fig.show()

In [ ]:
# Filter out rows with null age values and get unique age groups
valid_age_groups = df.dropna(subset=['INVAGE'])['INVAGE'].unique()

# Group by age group, light conditions, and visibility conditions, and count the occurrences of collisions
age_light_visibility_counts = df.groupby(['INVAGE', 'LIGHT', 'VISIBILITY']).size().reset_index(name='Collision Count')

# Filter only the rows with valid age groups
age_light_visibility_counts = age_light_visibility_counts[age_light_visibility_counts['INVAGE'].isin(valid_age_groups)]

# Create a grouped bar plot using Plotly
fig = px.bar(age_light_visibility_counts, x='INVAGE', y='Collision Count', color='VISIBILITY',
             facet_col='LIGHT', facet_col_wrap=3,
             title='Comparison of Light and Visibility Conditions by Age Group',
             labels={'Collision Count': 'Collision Count', 'INVAGE': 'Age Group', 'VISIBILITY': 'Visibility Condition'},
             barmode='group')

# Show the plot
fig.show()

In [ ]:
# Filter out rows with null age values and get unique age groups
valid_age_groups = df.dropna(subset=['INVAGE'])['INVAGE'].unique()

# Group by age group and count the occurrences of collisions
age_collision_counts = df['INVAGE'].value_counts().reset_index()
age_collision_counts.columns = ['Age Group', 'Collision Count']

# Filter only the rows with valid age groups
age_collision_counts = age_collision_counts[age_collision_counts['Age Group'].isin(valid_age_groups)]

# Create a bar plot using Plotly
fig = px.bar(age_collision_counts, x='Age Group', y='Collision Count',
             title='Collision Occurrences by Age Group',
             labels={'Collision Count': 'Collision Count', 'Age Group': 'Age Group'})

# Show the plot
fig.show()

In [ ]:
# Group by injury type and count the occurrences of collisions
injury_collision_counts = df['INJURY'].value_counts().reset_index()
injury_collision_counts.columns = ['Injury Type', 'Collision Count']

# Create a bar plot using Plotly
fig = px.bar(injury_collision_counts, x='Injury Type', y='Collision Count',
             title='Collision Occurrences by Injury Type',
             labels={'Collision Count': 'Collision Count', 'Injury Type': 'Injury Type'})

# Show the plot
fig.show()

In [ ]:
# Group by initial direction of travel and count the occurrences of collisions
initdir_collision_counts = df['INITDIR'].value_counts().reset_index()
initdir_collision_counts.columns = ['Initial Direction', 'Collision Count']

# Create a bar plot using Plotly
fig = px.bar(initdir_collision_counts, x='Initial Direction', y='Collision Count',
             title='Collision Occurrences by Initial Direction of Travel',
             labels={'Collision Count': 'Collision Count', 'Initial Direction': 'Initial Direction'})

# Show the plot
fig.show()

In [ ]:
# Group by driver condition and count the occurrences of collisions
drivcond_collision_counts = df['DRIVCOND'].value_counts().reset_index()
drivcond_collision_counts.columns = ['Driver Condition', 'Collision Count']

# Create a bar plot using Plotly
fig = px.bar(drivcond_collision_counts, x='Driver Condition', y='Collision Count',
             title='Collision Occurrences by Driver Condition',
             labels={'Collision Count': 'Collision Count', 'Driver Condition': 'Driver Condition'})

# Show the plot
fig.show()

In [ ]:
# Filter out rows with missing vehicle type
df_filtered = df.dropna(subset=['VEHTYPE'])

# Group by vehicle type and count the occurrences
collision_counts = df_filtered['VEHTYPE'].value_counts().reset_index()
collision_counts.columns = ['Vehicle Type', 'Collision Count']

# Create a bar chart using Plotly Express
fig = px.bar(collision_counts, x='Vehicle Type', y='Collision Count',
             title='Total Collision Comparison by Vehicle Type',
             labels={'Vehicle Type': 'Vehicle Type', 'Collision Count': 'Collision Count'})

# Show the plot
fig.show()

In [ ]:
# Filter out rows with missing pedestrian condition
df_filtered = df.dropna(subset=['PEDCOND'])

# Group by pedestrian condition and count the occurrences
collision_counts = df_filtered['PEDCOND'].value_counts().reset_index()
collision_counts.columns = ['Pedestrian Condition', 'Collision Count']

# Create a bar chart using Plotly Express
fig = px.bar(collision_counts, x='Pedestrian Condition', y='Collision Count',
             title='Total Collision Comparison by Pedestrian Condition',
             labels={'Pedestrian Condition': 'Pedestrian Condition', 'Collision Count': 'Collision Count'})

# Show the plot
fig.show()

In [ ]:
# Count the occurrences of each factor
counts = {
    'Cyclist Involved': df['CYCLIST'].notnull().sum(),
    'Pedestrian Involved': df['PEDESTRIAN'].notnull().sum(),
    'Transit or City Vehicle Involved': df['TRSN_CITY_VEH'].notnull().sum(),
    'Passenger Involved': df['PASSENGER'].notnull().sum(),
    'Speeding Related': df['SPEEDING'].notnull().sum(),
    'Red Light Related': df['REDLIGHT'].notnull().sum(),
    'Alcohol Related': df['ALCOHOL'].notnull().sum(),
}

# Create a DataFrame from the counts
data = pd.DataFrame.from_dict(counts, orient='index', columns=['Count']).reset_index()

# Rename columns
data.columns = ['Factor', 'Count']

# Create a bar chart using Plotly Express
fig = px.bar(data, x='Factor', y='Count',
             title='Total Accidents and Involvement of Different Factors',
             labels={'Factor': 'Factor', 'Count': 'Count'})

# Show the plot
fig.show()

In [ ]:
# Count the occurrences of each factor
counts = {
    'Cyclist Involved': df['CYCLIST'].notnull().sum(),
    'Pedestrian Involved': df['PEDESTRIAN'].notnull().sum(),
    'Transit or City Vehicle Involved': df['TRSN_CITY_VEH'].notnull().sum(),
    'Passenger Involved': df['PASSENGER'].notnull().sum(),
    'Speeding Related': df['SPEEDING'].notnull().sum(),
    'Red Light Related': df['REDLIGHT'].notnull().sum(),
    'Alcohol Related': df['ALCOHOL'].notnull().sum(),
}

# Create a DataFrame from the counts
data = pd.DataFrame.from_dict(counts, orient='index', columns=['Count']).reset_index()

# Rename columns
data.columns = ['Factor', 'Count']

# Create a pie chart using Plotly Express
fig = px.pie(data, names='Factor', values='Count',
             title='Distribution of Accidents by Factor')

# Show the plot
fig.show()

In [ ]:
# Group by involvement type and count the occurrences
collision_counts = df_filtered['INVTYPE'].value_counts().reset_index()
collision_counts.columns = ['Involvement Type', 'Collision Count']

# Create a bar chart using Plotly Express
fig = px.bar(collision_counts, x='Involvement Type', y='Collision Count',
             title='Total Collisions by Involvement Type',
             labels={'Involvement Type': 'Involvement Type', 'Collision Count': 'Collision Count'})

# Show the plot
fig.show()

In [ ]:
# Define a function to map dates to seasons
def get_season(date):
    month = date.month
    if 3 <= month <= 5:
        return 'Spring'
    elif 6 <= month <= 8:
        return 'Summer'
    elif 9 <= month <= 11:
        return 'Autumn'
    else:
        return 'Winter'

# Convert the 'DATE' column to datetime format if it's not already
df['DATE'] = pd.to_datetime(df['DATE'])

# Apply the function to create a new column for seasons
df['SEASON'] = df['DATE'].apply(get_season)

# Filter out rows with missing season information
df_filtered = df.dropna(subset=['SEASON'])

# Group by season and count the occurrences
season_counts = df_filtered['SEASON'].value_counts().reset_index()
season_counts.columns = ['Season', 'Accident Count']

# Create a bar chart using Plotly Express
fig = px.bar(season_counts, x='Season', y='Accident Count',
             title='Accidents by Season',
             labels={'Season': 'Season', 'Accident Count': 'Number of Accidents'})

# Show the plot
fig.show()

In [ ]:
# Filter out rows with missing season and district information
df_filtered = df.dropna(subset=['SEASON', 'DISTRICT'])

# Group by season and district and count the occurrences
season_district_counts = df_filtered.groupby(['DISTRICT', 'SEASON']).size().reset_index(name='Accident Count')

# Create a grouped bar chart using Plotly Express
fig = px.bar(season_district_counts, x='DISTRICT', y='Accident Count', color='SEASON',
             title='Accidents by District and Season',
             labels={'DISTRICT': 'District', 'Accident Count': 'Number of Accidents'})

# Show the plot
fig.show()

In [ ]:
# First result: Total null values for each feature
null_counts = df.isnull().sum()
null_df = pd.DataFrame(null_counts, columns=["Missing Values"])

# Second result: Total unique values for each feature
unique_values = df.nunique().reset_index()
unique_values.columns = ["Feature", "Unique Values"]

# Combine the two DataFrames
combined_df = pd.merge(unique_values, null_df, left_on="Feature", right_index=True)

# Display the combined DataFrame
print(combined_df)

              Feature  Unique Values  Missing Values
0              INDEX_          15000               0
1              ACCNUM           3822            3698
2                YEAR             13               0
3                DATE           3082               0
4                TIME           1276               0
5             STREET1           1547               0
6             STREET2           2344            1343
7              OFFSET            339           13072
8          ROAD_CLASS              9             357
9            DISTRICT              4              14
10            WARDNUM             71               0
11           LATITUDE           3475               0
12          LONGITUDE           3901               0
13           LOCCOORD              7              90
14             ACCLOC              9            5450
15           TRAFFCTL             10              29
16         VISIBILITY              8              14
17              LIGHT              9          

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'])

# Accessing year, month, and day components
df['Year'] = df['DATE'].dt.year
df['Month'] = df['DATE'].dt.month
df['Day'] = df['DATE'].dt.day

# Displaying the updated DataFrame
print(df[['DATE', 'Year', 'Month', 'Day']].head())

                       DATE  Year  Month  Day
0 2006-01-07 05:00:00+00:00  2006      1    7
1 2006-01-07 05:00:00+00:00  2006      1    7
2 2006-01-07 05:00:00+00:00  2006      1    7
3 2006-01-09 05:00:00+00:00  2006      1    9
4 2006-01-09 05:00:00+00:00  2006      1    9


In [ ]:
one_accident = df[df.duplicated(subset=[
    'YEAR',
    'DATE',
    'TIME',
    'STREET1',
    'WARDNUM',
    'LONGITUDE',
    'LATITUDE',
    'LIGHT',
    'HOOD_158',
    'NEIGHBOURHOOD_158',
    'HOOD_140',
    'NEIGHBOURHOOD_140',
    'DIVISION'
], keep=False)]

# Group by the condition and assign a unique index ID
one_accident['ACC_NUM'] = one_accident.groupby([
    'YEAR',
    'DATE',
    'TIME',
    'STREET1',
    'WARDNUM',
    'LONGITUDE',
    'LATITUDE',
    'LIGHT',
    'HOOD_158',
    'NEIGHBOURHOOD_158',
    'HOOD_140',
    'NEIGHBOURHOOD_140',
    'DIVISION'
]).ngroup()

# Merge the unique index IDs back to the original DataFrame
df = pd.merge(df, one_accident[['ACC_NUM']], left_index=True, right_index=True, how='left')

C:\Users\rajgo\AppData\Local\Temp\ipykernel_15996\3890231929.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df['ACC_NUM'].count()

14749

In [ ]:
df['ACC_NUM'].isna().sum()

251

In [ ]:
df[df['ACC_NUM'].isna()].head(30)

,INDEX_,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,SEASON,Year,Month,Day,ACC_NUM
96,3422670,905761.0,2006,2006-05-30 04:00:00+00:00,2328,ROSEDALE VALLEY RD,BAYVIEW AVE,NaN,Minor Arterial,Toronto and East York,1113,43.671645,-79.366090,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,25 to 29,Fatal,NaN,East,Motorcycle,Going Ahead,Lost control,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,71,Cabbagetown-South St.James Town,71,Cabbagetown-South St.James Town (71),D51,660,Spring,2006,5,30,NaN
120,3450204,916024.0,2006,2006-07-30 04:00:00+00:00,2155,F G GARDINER XY Ramp W,GARDINER W S KINGSWAY RAMP,NaN,NaN,Toronto and East York,4,43.634645,-79.471590,Exit Ramp Westbound,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,20 to 24,Fatal,NaN,West,Motorcycle,Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,South Parkdale,85,South Parkdale (85),D11,881,Summer,2006,7,30,NaN
161,3490523,930890.0,2006,2006-10-18 04:00:00+00:00,2136,OLD FINCH AVE,REESOR RD,NaN,Collector,Scarborough,25,43.824745,-79.190590,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,20 to 24,Fatal,NaN,West,Motorcycle,Going Ahead,Lost control,Inattentive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144,Morningside Heights,131,Rouge (131),D42,1142,Autumn,2006,10,18,NaN
309,4211231,987376.0,2007,2007-08-30 04:00:00+00:00,2252,F G GARDINER XY W,JAMESON AVE,NaN,NaN,Toronto and East York,4,43.632745,-79.433790,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,25 to 29,Fatal,NaN,West,Motorcycle,Going Ahead,Lost control,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85,South Parkdale,85,South Parkdale (85),D14,2559,Summer,2007,8,30,NaN
310,4273923,987500.0,2007,2007-09-02 04:00:00+00:00,48,SHEPPARD AVE W,SENTINEL RD,NaN,Major Arterial,North York,6,43.743645,-79.491790,Intersection,NaN,Traffic Signal,Clear,Dark,Dry,Fatal,SMV Other,Driver,40 to 44,Fatal,NaN,East,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,Downsview,26,Downsview-Roding-CFB (26),D31,2917,Autumn,2007,9,2,NaN
434,5363175,1039861.0,2008,2008-05-24 04:00:00+00:00,30,BAYVIEW Aven,BLOOR Ramp,NaN,NaN,Toronto and East York,11,43.677146,-79.367893,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,SMV Other,Motorcycle Driver,25 to 29,Fatal,21.0,North,Motorcycle,Going Ahead,Exceeding Speed Limit,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,98,Rosedale-Moore Park,98,Rosedale-Moore Park (98),D53,3904,Spring,2008,5,24,NaN
504,5412194,1061341.0,2008,2008-09-28 04:00:00+00:00,545,WILSON AVE,HIGHWAY 400 S,NaN,Major Arterial,Etobicoke York,7,43.718145,-79.521090,Intersection,At Intersection,No Control,Clear,Dark,Wet,Fatal,SMV Other,Driver,55 to 59,Fatal,42.0,East,"Automobile, Station Wagon",Going Ahead,Exceeding Speed Limit,Had Been Drinking,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,154,Oakdale-Beverley Heights,26,Downsview-Roding-CFB (26),D31,3991,Autumn,2008,9,28,NaN
961,7378164,1276186.0,2012,2012-01-20 05:00:00+00:00,2214,SENECA HILL DR,DON MILLS RD,NaN,Collector,North York,17,43.790045,-79.355190,Mid-Block,NaN,No Control,Snow,Dark,Packed Snow,Fatal,SMV Other,Driver,85 to 89,Fatal,4.0,West,"Automobile, Station Wagon",Going Ahead,Lost control,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,N

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df.drop("INDEX_", axis=1, inplace=True)

In [ ]:
df[df['ACC_NUM'].isna()].head(30)

,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,WARDNUM,LATITUDE,LONGITUDE,LOCCOORD,ACCLOC,TRAFFCTL,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,IMPACTYPE,INVTYPE,INVAGE,INJURY,FATAL_NO,INITDIR,VEHTYPE,MANOEUVER,DRIVACT,DRIVCOND,PEDTYPE,PEDACT,PEDCOND,CYCLISTYPE,CYCACT,CYCCOND,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,TRSN_CITY_VEH,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,SEASON,Year,Month,Day,ACC_NUM
31,885650.0,2006,2006-01-29 05:00:00+00:00,2048,F G GARDINER XY W W,KIPLING AVE,NaN,NaN,Etobicoke York,3,43.618245,-79.52439,Mid-Block,NaN,No Control,Rain,Dark,Wet,Fatal,Pedestrian Collisions,Driver,20 to 24,NaN,NaN,West,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160,Mimico-Queensway,17,Mimico (includes Humber Bay Shores) (17),D22,603,Winter,2006,1,29,NaN
32,885650.0,2006,2006-01-29 05:00:00+00:00,2048,F G GARDINER XY W W,KIPLING AVE,NaN,NaN,Etobicoke York,3,43.618245,-79.52439,Mid-Block,NaN,No Control,Rain,Dark,Wet,Fatal,Pedestrian Collisions,Pedestrian,15 to 19,Fatal,NaN,South,Other,NaN,NaN,NaN,Pedestrian hit at mid-block,Running onto Roadway,Unknown,NaN,NaN,NaN,Yes,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,160,Mimico-Queensway,17,Mimico (includes Humber Bay Shores) (17),D22,611,Winter,2006,1,29,NaN
45,891993.0,2006,2006-03-06 05:00:00+00:00,2222,F G GARDINER XY W,PARK LAWN RD,NaN,NaN,Etobicoke York,3,43.627745,-79.48149,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Sideswipe,Passenger,15 to 19,Fatal,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,161,Humber Bay Shores,17,Mimico (includes Humber Bay Shores) (17),D22,288,Spring,2006,3,6,NaN
46,891993.0,2006,2006-03-06 05:00:00+00:00,2222,F G GARDINER XY W,PARK LAWN RD,NaN,NaN,Etobicoke York,3,43.627745,-79.48149,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Sideswipe,Passenger,20 to 24,Minor,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,161,Humber Bay Shores,17,Mimico (includes Humber Bay Shores) (17),D22,289,Spring,2006,3,6,NaN
47,891993.0,2006,2006-03-06 05:00:00+00:00,2222,F G GARDINER XY W,PARK LAWN RD,NaN,NaN,Etobicoke York,3,43.627745,-79.48149,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Sideswipe,Driver,35 to 39,Minor,NaN,West,"Automobile, Station Wagon",Overtaking,Lost control,"Ability Impaired, Alcohol Over .08",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,161,Humber Bay Shores,17,Mimico (includes Humber Bay Shores) (17),D22,290,Spring,2006,3,6,NaN
48,891993.0,2006,2006-03-06 05:00:00+00:00,2222,F G GARDINER XY W,PARK LAWN RD,NaN,NaN,Etobicoke York,3,43.627745,-79.48149,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Sideswipe,Vehicle Owner,unknown,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,161,Humber Bay Shores,17,Mimico (includes Humber Bay Shores) (17),D22,291,Spring,2006,3,6,NaN
49,891993.0,2006,2006-03-06 05:00:00+00:00,2222,F G GARDINER XY W,PARK LAWN RD,NaN,NaN,Etobicoke York,3,43.627745,-79.48149,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Sideswipe,Driver,25 to 29,Minor,NaN,West,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,161,Humber Bay Shores,17,Mimico (includes Humber Bay Shores) (17),D22,292,Spring,2006,3,6,NaN
50,891993.0,2006,2006-03-06 05:00:00+00:00,2222,F G GARDINER XY W,PARK LAWN RD,NaN,NaN,Etobicoke York,3,43.627745,-79.48149,Mid-Block,NaN,No Control,Clear,Dark,Dry,Fatal,Sideswipe,Driver,40 to 44,NaN,NaN,West,"Automobile, Station Wagon",Going Ahead,Driving Properly,Normal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,Yes,NaN,161,Humber Bay Shores,17,Mimico (includes Humber Bay Shores) (17),D22,293,Spring,2006,3,6,NaN
51,891993

In [ ]:
!pip install klib


In [ ]:
import klib

In [ ]:
df.dtypes


INDEX_                             int64
ACCNUM                           float64
YEAR                               int64
DATE                 datetime64[ns, UTC]
TIME                               int64
STREET1                           object
STREET2                           object
OFFSET                            object
ROAD_CLASS                        object
DISTRICT                          object
WARDNUM                            int64
LATITUDE                         float64
LONGITUDE                        float64
LOCCOORD                          object
ACCLOC                            object
TRAFFCTL                          object
VISIBILITY                        object
LIGHT                             object
RDSFCOND                          object
ACCLASS                           object
IMPACTYPE                         object
INVTYPE                           object
INVAGE                            object
INJURY                            object
FATAL_NO        

In [ ]:
klib.convert_datatypes(df)


,INDEX_,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId,SEASON,Year,Month,Day,ACC_NUM
0,3363207,882024.0,2006,2006-01-07 05:00:00+00:00,2325,STEELES AVE E,NINTH LINE ST,NaN,Minor Arterial,Scarborough,...,Morningside Heights,131,Rouge (131),D42,167,Winter,2006,1,7,4477.0
1,3363208,882024.0,2006,2006-01-07 05:00:00+00:00,2325,STEELES AVE E,NINTH LINE ST,NaN,Minor Arterial,Scarborough,...,Morningside Heights,131,Rouge (131),D42,171,Winter,2006,1,7,4477.0
2,3363209,882024.0,2006,2006-01-07 05:00:00+00:00,2325,STEELES AVE E,NINTH LINE ST,NaN,Minor Arterial,Scarborough,...,Morningside Heights,131,Rouge (131),D42,173,Winter,2006,1,7,4477.0
3,3363414,882174.0,2006,2006-01-09 05:00:00+00:00,1435,KENNEDY RD,GLAMORGAN AVE,NaN,Major Arterial,Scarborough,...,Dorset Park,126,Dorset Park (126),D41,309,Winter,2006,1,9,2246.0
4,3363415,882174.0,2006,2006-01-09 05:00:00+00:00,1435,KENNEDY RD,GLAMORGAN AVE,NaN,Major Arterial,Scarborough,...,Dorset Park,126,Dorset Park (126),D41,314,Winter,2006,1,9,2246.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,81474608,NaN,2018,2018-04-26 04:00:00+00:00,1942,ISLINGTON AVE,ALBION RD,NaN,Major Arterial,Etobicoke York,...,Thistletown-Beaumond Heights,3,Thistletown-Beaumond Heights (3),D23,14996,Spring,2018,4,26,3679.0
14996,81474609,NaN,2018,2018-04-26 04:00:00+00:00,1942,ISLINGTON AVE,ALBION RD,NaN,Major Arterial,Etobicoke York,...,Thistletown-Beaumond Heights,3,Thistletown-Beaumond Heights (3),D23,14997,Spring,2018,4,26,3679.0
14997,81474610,NaN,2018,2018-04-26 04:00:00+00:00,1942,ISLINGTON AVE,ALBION RD,NaN,Major Arterial,Etobicoke York,...,Thistletown-Beaumond Heights,3,Thistletown-Beaumond Heights (3),D23,14998,Spring,2018,4,26,3679.0
14998,81474611,NaN,2018,2018-04-26 04:00:00+00:00,1942,ISLINGTON AVE,ALBION RD,NaN,Major Arterial,Etobicoke York,...,Thistletown-Beaumond Heights,3,Thistletown-Beaumond Heights (3),D23,14999,Spring,2018,4,26,3679.0


In [ ]:
df.dtypes

INDEX_                             int64
ACCNUM                           float64
YEAR                               int64
DATE                 datetime64[ns, UTC]
TIME                               int64
STREET1                           object
STREET2                           object
OFFSET                            object
ROAD_CLASS                        object
DISTRICT                          object
WARDNUM                            int64
LATITUDE                         float64
LONGITUDE                        float64
LOCCOORD                          object
ACCLOC                            object
TRAFFCTL                          object
VISIBILITY                        object
LIGHT                             object
RDSFCOND                          object
ACCLASS                           object
IMPACTYPE                         object
INVTYPE                           object
INVAGE                            object
INJURY                            object
FATAL_NO        

In [ ]:
# # Find null values in the 'ACC_NUM' column
# null_values_indices = df[df['ACC_NUM'].isnull()].index

# # Generate a sequence starting from 4545
# sequence = range(4545, 4545 + len(null_values_indices))

# # Fill null values with the generated sequence
# df.loc[null_values_indices, 'ACC_NUM'] = sequence

# # Display the DataFrame with filled null values
# df["ACC_NUM"].isna().sum()

0

In [ ]:
df.isnull().sum()

INDEX_                   0
ACCNUM                3698
YEAR                     0
DATE                     0
TIME                     0
STREET1                  0
STREET2               1343
OFFSET               13072
ROAD_CLASS             357
DISTRICT                14
WARDNUM                  0
LATITUDE                 0
LONGITUDE                0
LOCCOORD                90
ACCLOC                5450
TRAFFCTL                29
VISIBILITY              14
LIGHT                    0
RDSFCOND                19
ACCLASS                  0
IMPACTYPE                0
INVTYPE                 10
INVAGE                   0
INJURY                7189
FATAL_NO             14407
INITDIR               4498
VEHTYPE               2055
MANOEUVER             6514
DRIVACT               7575
DRIVCOND              7579
PEDTYPE              12540
PEDACT               12550
PEDCOND              12555
CYCLISTYPE           14365
CYCACT               14379
CYCCOND              14380
PEDESTRIAN            9034
C